<a href="https://colab.research.google.com/github/miweru/colab-experiments/blob/master/KERAS_POS_Tagging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Das hier sind unsere Trainingsdaten.
! git clone https://github.com/UniversalDependencies/UD_German-GSD.git
!ls

fatal: destination path 'UD_German-GSD' already exists and is not an empty directory.
sample_data  Savedvecs.gz  UD_German-GSD


In [2]:
!ls -l

total 23260
drwxr-xr-x 2 root root     4096 Dec  3 17:43 sample_data
-rw-r--r-- 1 root root 23805972 Dec  4 22:06 Savedvecs.gz
drwxr-xr-x 4 root root     4096 Dec  4 21:48 UD_German-GSD


In [0]:
for ln, line in enumerate(open("UD_German-GSD/de_gsd-ud-train.conllu")):
  if ln>100:
    break
  print(line)

# sent_id = train-s1

# text = Sehr gute Beratung, schnelle Behebung der Probleme, so stelle ich mir Kundenservice vor.

1	Sehr	sehr	ADV	ADV	_	2	advmod	_	_

2	gute	gut	ADJ	ADJA	Case=Nom|Gender=Fem|Number=Sing	3	amod	_	_

3	Beratung	Beratung	NOUN	NN	Case=Nom|Gender=Fem|Number=Sing	0	root	_	SpaceAfter=No

4	,	,	PUNCT	$,	_	6	punct	_	_

5	schnelle	schnell	ADJ	ADJA	Case=Nom|Gender=Fem|Number=Sing	6	amod	_	_

6	Behebung	Behebung	NOUN	NN	Case=Nom|Gender=Fem|Number=Sing	3	conj	_	_

7	der	der	DET	ART	Case=Gen|Definite=Def|Gender=Neut|Number=Plur|PronType=Art	8	det	_	_

8	Probleme	Problem	NOUN	NN	Case=Gen|Gender=Neut|Number=Plur	6	nmod	_	SpaceAfter=No

9	,	,	PUNCT	$,	_	11	punct	_	_

10	so	so	ADV	ADV	_	11	advmod	_	_

11	stelle	stellen	VERB	VVFIN	Mood=Ind|Number=Sing|Person=1|Tense=Pres|VerbForm=Fin	3	parataxis	_	_

12	ich	ich	PRON	PPER	Case=Nom|Number=Sing|Person=1|PronType=Prs	11	nsubj	_	_

13	mir	ich	PRON	PRF	Case=Dat|Number=Sing|Person=1|PronType=Prs|Reflex=Yes	11	iobj	_	_

14	Kundenservice	Ku

In [0]:
import pandas as pd
ddata=dict()
#data=dict()
for ln, line in enumerate(open("UD_German-GSD/de_gsd-ud-train.conllu")):
  if line.startswith("# sent_id"):
    sid=line.lstrip("# sent_id = ").strip()
    #data[sid]=dict()
    #sdata=data[sid]
  if line.startswith("#"):
    continue
  cont=line.split("\t")
  if len(cont)<2:
    continue
  if cont[3]=="_":
    continue
  #sdata[cont[0]]={"word":cont[1].lower(),"tag":cont[3]}
  ddata[sid+"_"+"{0:03}".format(int(cont[0]))]={"lineNumber":int(cont[0]),"word_l":cont[1].lower(),"tag":cont[3]}
  
for ln, line in enumerate(open("UD_German-GSD/de_gsd-ud-dev.conllu")):
  if line.startswith("# sent_id"):
    sid=line.lstrip("# sent_id = ").strip()
    #data[sid]=dict()
    #sdata=data[sid]
  if line.startswith("#"):
    continue
  cont=line.split("\t")
  if len(cont)<2:
    continue
  if cont[3]=="_":
    continue
  #sdata[cont[0]]={"word":cont[1].lower(),"tag":cont[3]}
  ddata[sid+"_"+"{0:03}".format(int(cont[0]))]={"lineNumber":int(cont[0]),"word_l":cont[1].lower(),"tag":cont[3]}

#eval=dict()
for ln, line in enumerate(open("UD_German-GSD/de_gsd-ud-test.conllu")):
  if line.startswith("# sent_id"):
    sid=line.lstrip("# sent_id = ").strip()
    #data[sid]=dict()
    #sdata=data[sid]
  if line.startswith("#"):
    continue
  cont=line.split("\t")
  if len(cont)<2:
    continue
  if cont[3]=="_":
    continue
  #sdata[cont[0]]={"word":cont[1].lower(),"tag":cont[3]}
  ddata[sid+"_"+"{0:03}".format(int(cont[0]))]={"lineNumber":int(cont[0]),"word_l":cont[1].lower(),"tag":cont[3]}
myframe=pd.DataFrame(ddata)

In [4]:
myframe

,-s100_001,-s100_002,-s100_003,-s100_004,-s100_005,-s100_006,-s100_007,-s100_008,-s100_009,-s100_010,...,v-s9_006,v-s9_007,v-s9_008,v-s9_009,v-s9_010,v-s9_011,v-s9_012,v-s9_013,v-s9_014,v-s9_015
lineNumber,1,2,3,4,5,6,7,8,9,10,...,6,7,8,9,10,11,12,13,14,15
tag,DET,NOUN,AUX,ADV,X,NOUN,AUX,ADV,ADJ,PUNCT,...,PRON,AUX,ADJ,ADP,DET,NOUN,CCONJ,DET,NOUN,PUNCT
word_l,die,ausstattung,ist,sehr,spartanisch.das,zimmer,war,sehr,klein,.,...,ich,bin,begeistert,von,der,auswahl,und,der,qualität,.


In [0]:
#from tensorflow import keras

In [0]:
#Gut, wandeln wir das mal in Sequenzen von Sätzen um:
target=[]
train=[]
md=[]
mt=[]

for ln in myframe:
  mo=myframe[ln]
  if mo["lineNumber"]==1:
    if md:
      target.append(mt)
      train.append(md)
    md=[]
    mt=[]
  md.append(mo["word_l"])
  mt.append(mo["tag"])
md.append(mo["word_l"])
mt.append(mo["tag"])

In [0]:
"""etarget=[]
etrain=[]
md=[]
mt=[]

for ln in evaluation:
  mo=evaluation[ln]
  if mo["lineNumber"]==1:
    if md:
      etarget.append(mt)
      etrain.append(md)
    md=[]
    mt=[]
  md.append(mo["word_l"])
  mt.append(mo["tag"])
etarget.append(mt)
etrain.append(md)"""

In [0]:
#del(myframe)

#del(evaluation)

In [0]:
#myframe[ln]

In [6]:
"""
Das hier sind Schritte, um das Modell so klein zu machen, dass es auch in den Arbeitsspeicher von Colab passt.
Dafür lade ich es, und schreibe jedes enthaltene Wort in einen neuen Vektor
Im nächsten durchlauf können wir dann die Embedding Matrix aus den Savedvecs erstellen


"""
if 1==1:
  !pip install gensim
  from gensim.models.keyedvectors import KeyedVectors 

  wordset={word for sentence in train for word in sentence}
  !wget https://s3-us-west-1.amazonaws.com/fasttext-vectors/word-vectors-v2/cc.de.300.vec.gz
  model=KeyedVectors.load_word2vec_format("cc.de.300.vec.gz",binary=False)
  !rm cc.de.300.* 
  mymodel=KeyedVectors(300)
  for word in wordset:
    if word in model:
      mymodel.add(word, model[word])
  del(model)
  mymodel.save("Savedvecs.gz")

    100% |████████████████████████████████| 23.6MB 1.6MB/s 
    100% |████████████████████████████████| 1.4MB 15.1MB/s 
    100% |████████████████████████████████| 133kB 26.7MB/s 
    100% |████████████████████████████████| 5.1MB 6.4MB/s 
    100% |████████████████████████████████| 61kB 21.6MB/s 
    100% |████████████████████████████████| 552kB 23.0MB/s 
  Running setup.py bdist_wheel for smart-open ... - \ done
  Stored in directory: /root/.cache/pip/wheels/23/00/44/e5b939f7a80c04e32297dbd6d96fa3065af89ecf57e2b5f89f
  Running setup.py bdist_wheel for bz2file ... - done
  Stored in directory: /root/.cache/pip/wheels/81/75/d6/e1317bf09bf1af5a30befc2a007869fa6e1f516b8f7c591cb9
Successfully built smart-open bz2file
--2018-12-05 10:03:23--  https://s3-us-west-1.amazonaws.com/fasttext-vectors/word-vectors-v2/cc.de.300.vec.gz
Resolving s3-us-west-1.amazonaws.com (s3-us-west-1.amazonaws.com)... 52.219.24.33
Connecting to s3-us-west-1.amazonaws.com (s3-us-west-1.amazonaws.com)|52.219.24

In [5]:
!pip install gensim

NameError: ignored

In [0]:
#mymodel.save("Savedvecs.gz")

In [0]:
!ls -l

In [0]:
#line

In [0]:
KDW

In [4]:
import keras

Using TensorFlow backend.


In [24]:

tokenizer=keras.preprocessing.text.Tokenizer(filter="") #NEU
tokenizer.fit_on_texts(train)
X1=tokenizer.texts_to_sequences(train)
#EX=tokenizer.texts_to_sequences(etrain)
#del(train)
#del(etrain)

TypeError: ignored

In [0]:
seqlength=max([len(item) for item in X1])

In [0]:
MAX_SEQUENCE_LENGTH=40
X1=keras.preprocessing.sequence.pad_sequences(X1, maxlen=MAX_SEQUENCE_LENGTH, padding="pre",truncating= 'post')
#EX=keras.preprocessing.sequence.pad_sequences(EX, maxlen=MAX_SEQUENCE_LENGTH, padding="pre",truncating= 'post')

In [8]:
X1.shape

(15589, 20)

In [0]:
input_baskets=[list() for i in range(MAX_SEQUENCE_LENGTH)]
#AuxilaryInputs
for text in train:
  for i in range(MAX_SEQUENCE_LENGTH):
    if len(text)<=i:
      input_baskets[i].append("_")
    else:
      input_baskets[i].append(text[i])
#Nun sollten alle baskets gleich lang sein

In [0]:
for basket in input_baskets:
  print(len(basket))

'_'

In [0]:
from keras.preprocessing.sequence import pad_sequences
aux_tokenizer=keras.preprocessing.text.Tokenizer(char_level=True,filters="") #Hier brauchen wir die buchstabeneinstellung
aux_tokenizer.fit_on_texts(train)
auxseq=[pad_sequences(aux_tokenizer.texts_to_sequences(basket), maxlen=20, padding="pre",truncating='pre') for basket in input_baskets]

In [11]:
auxseq[2][2]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0, 3444, 4954, 6730,  290,  832], dtype=int32)

In [12]:
len(aux_tokenizer.word_index)

50490

In [0]:
X1

In [0]:
targetTokenizer=keras.preprocessing.text.Tokenizer()
targetTokenizer.fit_on_texts(target)
Y=targetTokenizer.texts_to_sequences(target)
#EY=targetTokenizer.texts_to_sequences(etarget)

In [0]:
#ctarget=targetTokenizer.texts_to_sequences(target+etarget)

In [0]:
#Y=ctarget[:len(target)]
#EY=ctarget[len(target):]

In [36]:
Y.shape

AttributeError: ignored

In [25]:
numoftags=len({pos for sentence in target for pos in sentence})
print(numoftags)

16


In [14]:
#print(len({pos for sentence in etarget for pos in sentence}))

15


In [0]:
#Y=np.array([[i==num for i in range(numoftags)] for sentence in Y for num in sentence],dtype=np.bool)

In [0]:
#EY=np.array([[i==num for i in range(numoftags)] for sentence in EY for num in sentence],dtype=np.bool)

In [0]:
Y=keras.preprocessing.sequence.pad_sequences(Y, maxlen=MAX_SEQUENCE_LENGTH, padding="pre",truncating= 'post')
#EY=keras.preprocessing.sequence.pad_sequences(EY, maxlen=MAX_SEQUENCE_LENGTH, padding="pre",truncating= 'post')

In [0]:
import numpy as np

In [0]:
np.bool

In [0]:
from keras.utils import to_categorical

In [0]:
Y=to_categorical(Y, num_classes=17)
#EY=to_categorical(EY, num_classes=17)

In [40]:
Y.shape

(15589, 20, 17)

In [0]:
!ls -l

In [0]:
EY.shape

In [0]:
!pip install gensim

In [0]:
from gensim.models.keyedvectors import KeyedVectors
#import gensim.models.keyedvectors as kv

In [0]:
#!rm cc.de.300.*

In [0]:
#model=KeyedVectors.load_word2vec_format("cc.de.300.vec.gz",binary=False)
#model=KeyedVectors.load_word2vec_format("https://s3-us-west-1.amazonaws.com/fasttext-vectors/word-vectors-v2/cc.de.300.vec.gz",binary=False)

In [0]:
#del(model)

In [0]:
weights=np.random.random((5,30))

In [31]:
model

In [0]:

from gensim.models.keyedvectors import KeyedVectors 
gmodel=KeyedVectors.load("Savedvecs.gz")
weights=np.random.random((len(tokenizer.word_index)+1,300))
for token in tokenizer.word_index:
  if token in gmodel:
    weights[tokenizer.word_index[token]]=gmodel[token]
  

In [29]:
weights.shape

(50491, 300)

In [36]:
weights

array([[ 0.32606257,  0.31952443,  0.5724918 , ...,  0.61112887,
         0.14428032,  0.63668434],
       [-0.0257    ,  0.0093    , -0.0562    , ...,  0.0404    ,
        -0.0495    , -0.0133    ],
       [-0.0354    , -0.0088    , -0.018     , ..., -0.0606    ,
        -0.1096    ,  0.0739    ],
       ...,
       [ 0.85787064,  0.01113347,  0.19147647, ...,  0.44606168,
         0.5597976 ,  0.06696565],
       [ 0.8124987 ,  0.75546648,  0.32687834, ...,  0.02963436,
         0.53246644,  0.51197454],
       [ 0.0064    , -0.0098    ,  0.0024    , ...,  0.01      ,
        -0.0088    ,  0.0015    ]])

In [0]:
#embedding=model.get_keras_embedding()

In [0]:
from keras.models import Model

In [23]:
from keras.models import Model
from keras.layers import Conv1D, GlobalMaxPooling1D, Flatten
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, TimeDistributed, Bidirectional

sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
#embedded_sequences = embedding(sequence_input)
embedded_sequences=Embedding(50491,300, weights=[weights],mask_zero=True)(sequence_input)
x = Bidirectional(LSTM(100,return_sequences=True))(embedded_sequences)
x = Dropout(0.5)(x)
x = Bidirectional(LSTM(128,return_sequences=True))(x)
embed2= Embedding(50491, output_dim=700,mask_zero=True)(sequence_input)
x2= Bidirectional(LSTM(128,return_sequences=True))(embed2)
x2 = Dropout(0.5)(x2)
x2= Bidirectional(LSTM(128,return_sequences=True))(x2)
x = keras.layers.concatenate([x, x2])
x = Dropout(0.5)(x)
x = Bidirectional(LSTM(128,return_sequences=True))(x)
x = Dropout(0.5)(x)
x = Bidirectional(LSTM(64,return_sequences=True))(x)
preds = TimeDistributed(Dense(numoftags+1,activation="softmax"))(x)

model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['acc'])
model.fit(X1, Y, epochs=100, batch_size=256, validation_split=0.1)


Train on 14030 samples, validate on 1559 samples
Epoch 1/100
14030/14030 [==============================] - 59s 4ms/step - loss: 2.1248 - acc: 0.2847 - val_loss: 1.4884 - val_acc: 0.5040
Epoch 2/100
14030/14030 [==============================] - 49s 4ms/step - loss: 0.8932 - acc: 0.7182 - val_loss: 0.5492 - val_acc: 0.8394
Epoch 3/100
14030/14030 [==============================] - 49s 4ms/step - loss: 0.3788 - acc: 0.8904 - val_loss: 0.3376 - val_acc: 0.9025
Epoch 4/100
14030/14030 [==============================] - 49s 4ms/step - loss: 0.2363 - acc: 0.9355 - val_loss: 0.2868 - val_acc: 0.9171
Epoch 5/100
14030/14030 [==============================] - 49s 4ms/step - loss: 0.1768 - acc: 0.9518 - val_loss: 0.3219 - val_acc: 0.9059
Epoch 6/100
14030/14030 [==============================] - 49s 3ms/step - loss: 0.1470 - acc: 0.9588 - val_loss: 0.2731 - val_acc: 0.9218
Epoch 7/100
14030/14030 [==============================] - 49s 3ms/step - loss: 0.1247 - acc: 0.9647 - val_loss: 0.2995 - v

KeyboardInterrupt: ignored

In [0]:
"""from keras.models import Sequential
from keras.layers import Conv1D, GlobalMaxPooling1D, Flatten
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, TimeDistributed, Bidirectional

model = Sequential()
model.add(Input(dtype='int32',tensor=X1))
model.add(embedding)
model.add(Dropout(0.2))
model.add(Conv1D(300, 3, padding='valid',activation='relu',strides=2))
model.add(Conv1D(150, 3, padding='valid',activation='relu',strides=2))
model.add(Conv1D(75, 3, padding='valid',activation='relu',strides=2))
model.add(GlobalMaxPooling1D())
model.add(Dropout(0.2))
model.add(TimeDistributed(Dense(150,activation='sigmoid')))
model.add(Dropout(0.2))
model.add(TimeDistributed(Dense(numoftags,activation='sigmoid')))

model.compile(loss='categorical_crossentropy',optimizer='rmsprop',metrics=['acc'])

model.summary()"""

In [0]:
X1.max()

In [0]:
#keras.layers.TimeDistributed()
model_1 = Sequential()
model_1.add(Input((50,)))
model_1.add(embedding(X1))
model_1.add(Bidirectional(LSTM(256,return_sequences=True)))
model_1.add(Bidirectional(LSTM(256,return_sequences=True)))
model_1.add(TimeDistributed(Dense(250)))
model_1.add(Dropout(0.2))
model_1.add(Activation('relu'))
model_1.add(TimeDistributed(Dense(numoftags+1,activation="softmax")))
#model_1.add(Activation('sigmoid'))
model_1.compile(loss='categorical_crossentropy',optimizer='rmsprop',metrics=['acc'])
model_1.summary()

In [0]:

model.fit(x_train, y_train)#, validation_data=(x_val, y_val), epochs=2, batch_size=128)
score = model.evaluate(x_val, y_val, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [0]:
model_1.fit(X1, Y, validation_data=(EX, EY), epochs=100, batch_size=128)
score = model_1.evaluate(EX, EY, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [0]:
#for slice in Y:
 ä print(slice.shape)

In [0]:
X1

In [0]:
X1=tokenizer.sequences_to_matrix(train)

In [0]:
#Verwenden wir lieber mal testweise ein BiLSTM

In [62]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_23 (InputLayer)           (None, 20)           0                                            
__________________________________________________________________________________________________
input_24 (InputLayer)           (None, 20)           0                                            
__________________________________________________________________________________________________
input_25 (InputLayer)           (None, 20)           0                                            
__________________________________________________________________________________________________
input_26 (InputLayer)           (None, 20)           0                                            
__________________________________________________________________________________________________
input_27 (

In [0]:
#AUXMODEL

In [0]:
#del(model)

In [0]:
from keras.layers import RepeatVector

In [32]:
from keras.models import Model
from keras.layers import Conv1D, GlobalMaxPooling1D, Flatten
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, TimeDistributed, Bidirectional

sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences=Embedding(50491,300, weights=[weights],mask_zero=True)(sequence_input)
x = Bidirectional(LSTM(100,return_sequences=True))(embedded_sequences)
x = Dropout(0.5)(x)
x = Bidirectional(LSTM(100,return_sequences=True))(x) # ich habe die beiden mal deutlich reduziert

aux_inputs=[Input(shape=(20,), dtype="int32") for i in range(20)]
aux_embedding=Embedding(len(aux_tokenizer.word_index),50,mask_zero=True)
aux_embedded=[aux_embedding(inp) for inp in aux_inputs]
aux_lstm=LSTM(100)#,return_sequences=True)
aux_x = [Dropout(0.3)(aux_lstm(vals)) for vals in aux_embedded]
x2=keras.layers.concatenate(aux_x)
x2=Dense(4000)(x2)
x2=Dense(300)(x2)
x2=RepeatVector(20)(x2)
#x2=Bidirectional(LSTM(128,return_sequences=True))(x2)
x = keras.layers.concatenate([x, x2])
#x=x2
x = Dropout(0.5)(x)
x = Bidirectional(LSTM(256,return_sequences=True))(x)
x = Dropout(0.5)(x)
x = Bidirectional(LSTM(128,return_sequences=True))(x)
preds = TimeDistributed(Dense(numoftags+1,activation="softmax"))(x)

model = Model([sequence_input]+aux_inputs, preds)
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['acc'])
model.fit([X1]+auxseq, Y, epochs=100, batch_size=128, validation_split=0.1)


ValueError: ignored

In [32]:
[1]+[2,3entstehen]

[1, 2, 3]

In [0]:
#Wir implementieren in der Folge 2 Architekturen:
"""
 Die 50dim 20LSTMs beenden ihre Sequenzen und kippen das Ergebnis in einen großen Vektor
1. Nun verwenden wir den konkatenierten Vektor,
2.

"""
#PS: Wenn das da oben konvergiert, bastle ich ein Teilmodell, das ich dann in einen Vektor stecke, den ich dann einfriere
#Dieser Vektor wird dann in einem weiteren Modell als zusätzlicher Layer verwendet

In [0]:
from keras.layers import Reshape

In [51]:
#Version 2
from keras.models import Model
from keras.layers import Conv1D, GlobalMaxPooling1D, Flatten
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, TimeDistributed, Bidirectional, CuDNNLSTM
from keras.layers.merge import add

sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences=Embedding(50491,300, weights=[weights])(sequence_input)
x = Dropout(0.75)(embedded_sequences)
xn = Bidirectional(CuDNNLSTM(128,return_sequences=True))(x)
x = Dropout(0.75)(xn)
x = Bidirectional(CuDNNLSTM(100,return_sequences=True))(x) # ich habe die beiden mal deutlich reduziert

aux_inputs=[Input(shape=(20,), dtype="int32") for i in range(MAX_SEQUENCE_LENGTH)]
aux_embedding=Embedding(len(aux_tokenizer.word_index),30)
aux_embedded=[aux_embedding(inp) for inp in aux_inputs]
aux_lstm=CuDNNLSTM(200)#,return_sequences=True)
aux_x = [Dropout(0.3)(aux_lstm(vals)) for vals in aux_embedded]
dense=Dense(200)
aux_x = [dense(vals) for vals in aux_x]
x2=keras.layers.concatenate(aux_x) #Über welche Achse machen wir das denn?
x2=Reshape((MAX_SEQUENCE_LENGTH,200))(x2)
x2=Dropout(0.4)(x2)
x2=CuDNNLSTM(200, return_sequences=True)(x2)
x2=Bidirectional(CuDNNLSTM(200,return_sequences=True))(x2)
x = keras.layers.concatenate([x, x2])
#x=x2
x = Dropout(0.5)(x)
x = Bidirectional(CuDNNLSTM(256,return_sequences=True))(x)
x = Dropout(0.5)(x)
x = Bidirectional(CuDNNLSTM(128,return_sequences=True))(x)
x = add([x,xn])
preds = TimeDistributed(Dense(numoftags+1,activation="softmax"))(x)

model = Model([sequence_input]+aux_inputs, preds)
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['acc'])
hist=model.fit([X1]+auxseq, Y, epochs=100, batch_size=128, validation_split=0.1)


Train on 14030 samples, validate on 1559 samples
Epoch 1/100
14030/14030 [==============================] - 60s 4ms/step - loss: 1.1316 - acc: 0.6505 - val_loss: 0.4452 - val_acc: 0.8726
Epoch 2/100
14030/14030 [==============================] - 30s 2ms/step - loss: 0.3483 - acc: 0.8919 - val_loss: 0.2242 - val_acc: 0.9282
Epoch 3/100
14030/14030 [==============================] - 30s 2ms/step - loss: 0.2347 - acc: 0.9249 - val_loss: 0.1916 - val_acc: 0.9400
Epoch 4/100
14030/14030 [==============================] - 30s 2ms/step - loss: 0.1981 - acc: 0.9368 - val_loss: 0.1710 - val_acc: 0.9446
Epoch 5/100
14030/14030 [==============================] - 30s 2ms/step - loss: 0.1761 - acc: 0.9437 - val_loss: 0.1633 - val_acc: 0.9477
Epoch 6/100
14030/14030 [==============================] - 30s 2ms/step - loss: 0.1603 - acc: 0.9490 - val_loss: 0.1539 - val_acc: 0.9518
Epoch 7/100
14030/14030 [==============================] - 30s 2ms/step - loss: 0.1482 - acc: 0.9529 - val_loss: 0.1498 - v

KeyboardInterrupt: ignored

In [0]:
"""
model.add(Conv1D(300, 3, padding='valid',activation='relu',strides=2))
model.add(Conv1D(150, 3, padding='valid',activation='relu',strides=2))
model.add(Conv1D(75, 3, padding='valid',activation='relu',strides=2))
model.add(GlobalMaxPooling1D())
"""

In [0]:
# Kombinieren wir das mal mit Proessor Everts Produkt und führen es auf dme englishcne Tagger aus

In [52]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_195 (InputLayer)          (None, 20)           0                                            
__________________________________________________________________________________________________
input_196 (InputLayer)          (None, 20)           0                                            
__________________________________________________________________________________________________
input_197 (InputLayer)          (None, 20)           0                                            
__________________________________________________________________________________________________
input_198 (InputLayer)          (None, 20)           0                                            
__________________________________________________________________________________________________
input_199 

In [54]:
#Version 2 Tweaked? Mehr bilstms und mehr Dropout
from keras.models import Model
from keras.layers import Conv1D, GlobalMaxPooling1D, Flatten
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, TimeDistributed, Bidirectional, CuDNNLSTM
from keras.layers.merge import add

sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences=Embedding(50491,300, weights=[weights])(sequence_input)
x = Dropout(0.77)(embedded_sequences)
xn = Bidirectional(CuDNNLSTM(128,return_sequences=True))(x)
x = Dropout(0.77)(xn)
x = Bidirectional(CuDNNLSTM(128,return_sequences=True))(x) # ich habe die beiden mal deutlich reduziert

aux_inputs=[Input(shape=(20,), dtype="int32") for i in range(MAX_SEQUENCE_LENGTH)]
aux_embedding=Embedding(len(aux_tokenizer.word_index),30)
aux_embedded=[aux_embedding(inp) for inp in aux_inputs]
aux_lstm=Bidirectional(CuDNNLSTM(200))#,return_sequences=True)
aux_x = [Dropout(0.4)(aux_lstm(vals)) for vals in aux_embedded]
dense=Dense(200)
aux_x = [dense(vals) for vals in aux_x]
x2=keras.layers.concatenate(aux_x) #Über welche Achse machen wir das denn?
x2=Reshape((MAX_SEQUENCE_LENGTH,200))(x2)
x2=Dropout(0.6)(x2)
x2=Bidirectional(CuDNNLSTM(200, return_sequences=True))(x2)
x2=Dropout(0.6)(x2)
x2=Bidirectional(CuDNNLSTM(200,return_sequences=True))(x2)
x = keras.layers.concatenate([x, x2])
#x=x2
x = Dropout(0.6)(x)
x = Bidirectional(CuDNNLSTM(256,return_sequences=True))(x)
x = Dropout(0.55)(x)
x = Bidirectional(CuDNNLSTM(128,return_sequences=True))(x)
x = add([x,xn])
preds = TimeDistributed(Dense(numoftags+1,activation="softmax"))(x)

model = Model([sequence_input]+aux_inputs, preds)
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['acc'])
hist=model.fit([X1]+auxseq, Y, epochs=25, batch_size=512, validation_split=0.1)


Train on 14030 samples, validate on 1559 samples
Epoch 1/25
14030/14030 [==============================] - 59s 4ms/step - loss: 1.8645 - acc: 0.3994 - val_loss: 1.4587 - val_acc: 0.5486
Epoch 2/25
14030/14030 [==============================] - 25s 2ms/step - loss: 1.2234 - acc: 0.6358 - val_loss: 0.8579 - val_acc: 0.7540
Epoch 3/25
14030/14030 [==============================] - 25s 2ms/step - loss: 0.6901 - acc: 0.8062 - val_loss: 0.4522 - val_acc: 0.8755
Epoch 4/25
14030/14030 [==============================] - 25s 2ms/step - loss: 0.4405 - acc: 0.8713 - val_loss: 0.3182 - val_acc: 0.9046
Epoch 5/25
14030/14030 [==============================] - 25s 2ms/step - loss: 0.3228 - acc: 0.9029 - val_loss: 0.2508 - val_acc: 0.9267
Epoch 6/25
14030/14030 [==============================] - 25s 2ms/step - loss: 0.2634 - acc: 0.9193 - val_loss: 0.2171 - val_acc: 0.9328
Epoch 7/25
14030/14030 [==============================] - 25s 2ms/step - loss: 0.2294 - acc: 0.9290 - val_loss: 0.2186 - val_acc:

In [0]:
#Version 2 Tweaked? Mehr bilstms und mehr Dropout. Und ein bisschen komisches
from keras.models import Model
from keras.layers import Conv1D, GlobalMaxPooling1D, Flatten
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, TimeDistributed, Bidirectional, CuDNNLSTM
from keras.layers.merge import add

sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences=Embedding(50491,300, weights=[weights])(sequence_input)
x = Dropout(0.77)(embedded_sequences)
xn = Bidirectional(CuDNNLSTM(128,return_sequences=True))(x)
x = Dropout(0.77)(xn)
x = Bidirectional(CuDNNLSTM(128,return_sequences=True))(x) # ich habe die beiden mal deutlich reduziert

aux_inputs=[Input(shape=(20,), dtype="int32") for i in range(MAX_SEQUENCE_LENGTH)]
aux_embedding=Embedding(len(aux_tokenizer.word_index),30)
aux_embedded=[aux_embedding(inp) for inp in aux_inputs]
aux_lstm=Bidirectional(CuDNNLSTM(200,return_sequences=True))
aux_x = [Dropout(0.5)(aux_lstm(vals)) for vals in aux_embedded]
aux_lstme=Bidirectional(CuDNNLSTM(200))
aux_x = [Dropout(0.5)(aux_lstme(vals)) for vals in aux_x]
dense=Dense(200)
aux_x = [dense(vals) for vals in aux_x]
x2=keras.layers.concatenate(aux_x) #Über welche Achse machen wir das denn?
x2=Reshape((MAX_SEQUENCE_LENGTH,200))(x2)
x2=Dropout(0.6)(x2)
x2=Bidirectional(CuDNNLSTM(200, return_sequences=True))(x2)
x2=Dropout(0.6)(x2)
x2=Bidirectional(CuDNNLSTM(200,return_sequences=True))(x2)
#x = keras.layers.concatenate([x, x2])
x=x2 #Nun trainieren wir das Modell allein auf den Repräsentationen 
x = Dropout(0.6)(x)
x = Bidirectional(CuDNNLSTM(256,return_sequences=True))(x)
x = Dropout(0.55)(x)
x = Bidirectional(CuDNNLSTM(128,return_sequences=True))(x)
#x = add([x,xn])
preds = TimeDistributed(Dense(numoftags+1,activation="softmax"))(x)

model = Model([sequence_input]+aux_inputs, preds)
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['acc'])
histi=model.fit([X1]+auxseq, Y, epochs=30, batch_size=256, validation_split=0.1)

Train on 14030 samples, validate on 1559 samples
Epoch 1/30
14030/14030 [==============================] - 105s 8ms/step - loss: 2.3705 - acc: 0.2709 - val_loss: 2.1109 - val_acc: 0.3596
Epoch 2/30
14030/14030 [==============================] - 61s 4ms/step - loss: 2.1830 - acc: 0.3164 - val_loss: 1.9581 - val_acc: 0.3802
Epoch 3/30
14030/14030 [==============================] - 61s 4ms/step - loss: 1.9483 - acc: 0.3630 - val_loss: 1.7870 - val_acc: 0.4241
Epoch 4/30
14030/14030 [==============================] - 61s 4ms/step - loss: 1.8707 - acc: 0.3760 - val_loss: 1.7356 - val_acc: 0.4326
Epoch 5/30
14030/14030 [==============================] - 61s 4ms/step - loss: 1.8281 - acc: 0.3874 - val_loss: 1.6722 - val_acc: 0.4499
Epoch 6/30
14030/14030 [==============================] - 61s 4ms/step - loss: 1.6854 - acc: 0.4246 - val_loss: 1.5765 - val_acc: 0.4680
Epoch 7/30
14030/14030 [==============================] - 61s 4ms/step - loss: 1.6216 - acc: 0.4381 - val_loss: 1.6359 - val_acc